## 09 FAISS、HNSW还是BM25？如何选择最适合业务的 向量检索引擎？如何选择最适合业务的 向量检索引擎？

在 RAG（Retrieval-Augmented Generation）系统中，核心挑战在于：
如何在毫秒级延迟内，精准召回最有助于 LLM 回答问题的 top-k 文本片段。 

![rag](./09/output.png)


几个典型误区：
- “向量检索一定优于关键词匹配”
- “HNSW 一定比 FAISS Flat 快”
- “BM25 已经过时，可以直接淘汰”

温馨提示 ：本讲聚焦于“第一跳”检索引擎的选型决策。关于检索质量评估、混合打分融合、查询扩展等进阶内容，我们将在第10～16讲中深入讲解，此处仅作简要提及，避免一次信息过载。 

## 一、三种检索引擎的核心能力矩阵
| 能力维度         | BM25               | FAISS-IVF                          | HNSW                     |
|------------------|--------------------|------------------------------------|--------------------------|
| 内存/显存        | 低                 | 中                                 | 高                       |
| 可解释性         | 关键词高亮         | 弱                                 | 弱                       |
| 增量增删         | 秒级               | 分钟级（需 re-train IVF）          | 秒级（标记删除）         |
| 跨语言同义词     | 差                 | 好                                 | 好                       |
| 典型场景         | FAQ、政策条文      | 大规模新闻推荐                    | 电商搜索、客服           |
| 核心能力         | 关键词匹配         | 向量近似搜索                      | 图结构索引               |
| 优势领域         | 冷启动快、可解释性强、CPU 友好 | 高性能、支持百亿级向量、GPU 加速 | 高召回率、低延迟         |
| 局限性           | 无法处理语义相似性、同义词、拼写变体 | 动态更新难、构建成本高    | 内存占用高、索引构建慢   |

### 1.1 BM25：基于词频统计的经典方法
概念 ：BM25 是一种改进版的 TF-IDF 排序函数，通过词频（TF）、文档长度归一化和逆文档频率（IDF）加权，计算查询与文档的相关性得分。
优点 ：
- 冷启动友好：无需训练模型，部署门槛低；
- 可解释性强：可直接查看命中关键词及权重分布；
- 资源消耗低：纯 CPU 运行即可满足毫秒级响应。

缺点 ：
- 无法识别同义词、拼写错误或多语言表达；
- 长文本中关键词密度被稀释，影响评分准确性；
- 对用户输入模糊或泛化的问题适应性差。

适合场景 ：
- 语料高度结构化（如政策、法律条文）；
- 上线周期短、来不及训练模型；
- 需要向业务方解释“为何排第一”。
- 知识图谱问答，或传统搜索引擎中注重关键词匹配的场景。

### 1.2 FAISS：高效向量检索的工业首选
简介 ：由 Facebook 开发的高效向量相似度搜索库，广泛用于大规模语义检索任务。

核心能力 ：
- 提供多种索引结构（Flat、IVF、PQ、HNSW），适配不同精度与性能需求；
- 支持 GPU 加速，适用于超大规模向量数据集（百亿级）；
- 灵活配置：可在召回率、延迟与内存占用之间灵活平衡。

适合场景 ：
- 图像检索、推荐系统、语义问答等需高维向量匹配的场景；
- 对召回质量要求较高、具备一定算力预算的企业级应用。
- 大规模新闻推荐、内容去重、相似图片检索等对性能和规模有较高要求的场景。

### 1.3 HNSW：图结构索引，高召回语义检索首选
简介 ：一种基于图结构的近邻搜索算法，其复杂度接近 O(log n)实现高效检索。

核心能力 ：
- 高召回率接近暴力搜索（Brute-force）；
- 查询延迟低，适合实时检索场景；
- 多种开源实现成熟，如 faiss-hnswlib、Milvus、Qdrant、Weaviate。

局限性 ：
- 构建索引耗时较长；
- 内存占用高于其他索引结构；
- 不适合高频增删场景。

适合场景 ：
- 对召回率和延迟都有较高要求的语义检索；
- 数据规模百万级以上，且有实时性需求；
- 可接受较高内存开销的高性能服务。
- 电商搜索、客服、智能问答机器人、代码相似性搜索等需要高精度和低延迟的场景。

注意：HNSW的参数调优、压缩策略与构建细节将在第11讲深入讲解，本节我们聚焦于基本使用方法与工程部署建议。

## 二、实战：构建统一语料下的三种检索引擎原型

### 2.1 BM25 基线实现

In [6]:
import os
from dotenv import load_dotenv

load_dotenv()  # loads .env in current working directory

True

'/Users/linghuang/Git/LLMs/LLM-RAG/09'

In [9]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

# --- 1) LLM + Embeddings (LangChain) ---
openai_api_key = os.getenv("OPENAI_API_KEY")

llm = ChatOpenAI(model="gpt-4.1-nano-2025-04-14", temperature=0)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# --- 2) Load documents from a directory 
# This loads text files under ./09/ recursively by default.
cwd = os.getcwd()
loader = DirectoryLoader(
    cwd,
    glob="data.txt",
    loader_cls=TextLoader,
    loader_kwargs={"encoding": "utf-8"},
)
documents = loader.load()

# --- 3) Split into chunks (like SentenceSplitter(chunk_size=512)) ---
# SentenceSplitter is sentence-aware; in LangChain, the closest common default is RecursiveCharacterTextSplitter.
# Note: chunk_size is in characters here (not tokens).
splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=50,
)
chunks = splitter.split_documents(documents)

print("docs:", len(documents))
print("chunks:", len(chunks))
print("sample chunk:\n", chunks[0].page_content[:300])


docs: 1
chunks: 29
sample chunk:
 Paul Graham traces how his interests evolved from writing short stories and early programming (punch-card Fortran on an IBM 1401) to microcomputers, where interactive programming hooked him. In college he initially pursued philosophy, found it unsatisfying, and pivoted to AI, learning Lisp and even 


In [ ]:
print(chunks)

[TextNode(id_='6f11a4c3-80a5-4c82-a12c-f473132aa474', embedding=None, metadata={'file_path': '/Users/wilson/工作文档/rag 调优 50 讲/playbook/09/data.txt', 'file_name': 'data.txt', 'file_type': 'text/plain', 'file_size': 75041, 'creation_date': '2025-07-14', 'last_modified_date': '2025-07-14'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='5e6ef38f-2527-495c-b1c7-51c15dda049c', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_path': '/Users/wilson/工作文档/rag 调优 50 讲/playbook/09/data.txt', 'file_name': 'data.txt', 'file_type': 'text/plain', 'file_size': 75041, 'creation_date': '2025-07-14', 'last_modified_date': '2025-07-14'}, hash='a593f154975f49e38a1ef0d34c333849d0c30df928e0871d339774747363b04a'), <NodeRelati

In [ ]:
from langchain_community.retrievers import BM25Retriever
from langchain_core.documents import Document
import Stemmer

stemmer = Stemmer.Stemmer("english")
bm25 = BM25Retriever.from_documents(
    documents,
    k=5,
    stemmer=stemmer,
)


In [19]:
stemmer = Stemmer.Stemmer("english")
bm25 = BM25Retriever.from_documents(
    documents,
    k=5,
    stemmer=stemmer,
)

query = "What happened at Viaweb and Interleaf?"

# BM25 results assumed to come from LangChain BM25Retriever
bm25_results = bm25.invoke(query)

print(f"BM25 return results {len(bm25_results)} documents")
print(bm25_results[0].page_content)

BM25 return results 1 documents
Paul Graham traces how his interests evolved from writing short stories and early programming (punch-card Fortran on an IBM 1401) to microcomputers, where interactive programming hooked him. In college he initially pursued philosophy, found it unsatisfying, and pivoted to AI, learning Lisp and even reverse-engineering SHRDLU—until graduate school convinced him that symbolic AI of the era was fundamentally limited. He then doubled down on Lisp and building things, writing On Lisp, while also growing dissatisfied with how quickly software becomes obsolete.

A turning point came when he realized art can last, leading him to study painting (RISD, Florence) and juggle identities as a CS PhD student, Lisp hacker, and aspiring artist. After returning to the U.S. and working at Interleaf, he absorbed practical lessons about product, culture, and the idea that “low end eats high end.” He later co-founded Viaweb, discovering early the power of web apps (server-sid

### 2.2 FAISS IVF：高效向量索引实现

FAISS 是当前最主流的向量检索库之一，支持多种索引结构。它适用于从科研验证到工业部署的各种场景。
 FAISS 支持四种索引类型，我们以默认推荐的 IVF（Inverted File） 索引为例，兼顾召回率与查询效率，演示其构建与查询流程。

In [21]:
import os
from typing import List, Tuple

from dotenv import load_dotenv
load_dotenv()

from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.retrievers import BM25Retriever
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document

import Stemmer

stemmer = Stemmer.Stemmer("english")


def preview(doc: Document, n: int = 500) -> str:
    text = (doc.page_content or "").replace("\n", " ").strip()
    return text[:n] + ("..." if len(text) > n else "")

def print_results(title: str, docs: List[Document], n_preview: int = 500) -> None:
    print(f"\n{title} (count={len(docs)})")
    for i, d in enumerate(docs, 1):
        src = (d.metadata or {}).get("source", "")
        print("-" * 80)
        print(f"[{i}] source={src}")
        print(preview(d, n=n_preview))

def retrieve(retriever, query: str) -> List[Document]:
    try:
        return retriever.invoke(query)          # newer LangChain
    except Exception:
        return retriever.get_relevant_documents(query)  # older LangChain

# 1) Load
file_path = os.path.join(os.getcwd(), "data.txt")
documents = TextLoader(file_path, encoding="utf-8").load()

# 2) Split
splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=50)
docs = splitter.split_documents(documents)

# 3) BM25
bm25_retriever = BM25Retriever.from_documents(docs, k=2, stemmer=stemmer)

# 4) FAISS (correct way)
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
vectorstore = FAISS.from_documents(docs, embeddings)
faiss_retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

# 5) Query
query = "What happened at Viaweb and Interleaf?"
bm25_results = retrieve(bm25_retriever, query)
faiss_results = retrieve(faiss_retriever, query)

print_results("BM25 Results", bm25_results)
print_results("FAISS Results", faiss_results)

# 6) FAISS distances (lower = more similar)
faiss_with_scores: List[Tuple[Document, float]] = vectorstore.similarity_search_with_score(query, k=2)
if faiss_with_scores:
    avg_dist = sum(dist for _, dist in faiss_with_scores) / len(faiss_with_scores)
    print(f"\nFAISS average L2 distance: {avg_dist:.4f}")



BM25 Results (count=2)
--------------------------------------------------------------------------------
[1] source=/Users/linghuang/Git/LLMs/LLM-RAG/09/data.txt
[10] This was the first instance of what is now a familiar experience, and so was what happened next, when I read the comments and found they were full of angry people. How could I claim that Lisp was better than other languages? Weren't they all Turing complete? People who see the responses to essays I write sometimes tell me how sorry they feel for me, but I'm not exaggerating when I reply that it has always been like this, since the very beginning. It comes with the territory. An essay must t...
--------------------------------------------------------------------------------
[2] source=/Users/linghuang/Git/LLMs/LLM-RAG/09/data.txt
I picked orange as our color partly because it's the warmest, and partly because no VC used it. In 2005 all the VCs used staid colors like maroon, navy blue, and forest green, because they were tr

### 2.3 HNSW：高召回图索引实现

HNSW (Hierarchical Navigable Small World) 是一种基于图结构的近邻搜索算法，其核心思想是将向量空间构建成一个多层次的小世界网络。在搜索时，它从高层快速“跳跃”到目标区域，再逐层精确查找，复杂度接近 O(log n)，非常适合对召回率和延迟都有较高要求的场景。

In [ ]:
import time
import numpy as np
import hnswlib
from typing import List

from langchain_core.documents import Document
from langchain_community.retrievers import BM25Retriever
from langchain_openai import OpenAIEmbeddings

# ----------------------------
# 0) Inputs you must have
# ----------------------------
# docs: List[Document] = ...  # your split documents
# Example: docs = splitter.split_documents(loader.load())

TOP_K = 2
EMBED_DIM = 1536  # text-embedding-3-small
QUERY = "What happened at Viaweb and Interleaf?"

# ----------------------------
# 1) BM25 Retriever (baseline)
# ----------------------------
bm25_retriever = BM25Retriever.from_documents(docs, k=TOP_K)

# ----------------------------
# 2) HNSW Retriever (hnswlib)
# ----------------------------
class SimpleHNSWRetriever:
    """
    Minimal HNSW retriever for LangChain Documents using hnswlib.

    Notes:
    - space='cosine' in hnswlib returns distance = 1 - cosine_similarity
    - hnswlib expects float32 numpy arrays
    """

    def __init__(
        self,
        docs: List[Document],
        embeddings: OpenAIEmbeddings,
        top_k: int = 2,
        M: int = 16,
        ef_construction: int = 100,
        ef_search: int = 50,
    ):
        self.docs = docs
        self.embeddings = embeddings
        self.top_k = top_k

        # Build ID mapping
        self.id_to_doc = {i: d for i, d in enumerate(docs)}

        # Embed all documents once
        texts = [d.page_content for d in docs]
        vecs = embeddings.embed_documents(texts)  # List[List[float]]
        vecs = np.array(vecs, dtype=np.float32)

        # Safety check (dimension)
        if vecs.ndim != 2 or vecs.shape[1] != EMBED_DIM:
            raise ValueError(f"Expected embeddings shape (N, {EMBED_DIM}), got {vecs.shape}")

        # Build HNSW index
        self.index = hnswlib.Index(space="cosine", dim=EMBED_DIM)
        self.index.init_index(max_elements=len(docs), M=M, ef_construction=ef_construction)
        self.index.add_items(vecs, np.arange(len(docs)))
        self.index.set_ef(ef_search)  # higher => better recall, slower

    def retrieve(self, query: str) -> List[Document]:
        q = np.array([self.embeddings.embed_query(query)], dtype=np.float32)  # (1, dim)
        ids, dists = self.index.knn_query(q, k=self.top_k)

        # Attach hnsw distance as metadata (optional)
        results = []
        for idx, dist in zip(ids[0].tolist(), dists[0].tolist()):
            doc = self.id_to_doc[idx]
            # copy metadata so we don't mutate original docs globally
            meta = dict(doc.metadata or {})
            meta["hnsw_distance"] = float(dist)  # 1 - cosine_sim
            results.append(Document(page_content=doc.page_content, metadata=meta))
        return results


embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
hnsw_retriever = SimpleHNSWRetriever(docs, embeddings, top_k=TOP_K)

# ----------------------------
# 3) Performance Compare
# ----------------------------
def retrieve_lc(retriever, query: str) -> List[Document]:
    # LangChain-version-safe retrieval
    try:
        return retriever.invoke(query)
    except Exception:
        return retriever.get_relevant_documents(query)

# BM25
t0 = time.time()
bm25_docs = retrieve_lc(bm25_retriever, QUERY)
bm25_time = time.time() - t0

# HNSW
t0 = time.time()
hnsw_docs = hnsw_retriever.retrieve(QUERY)
hnsw_time = time.time() - t0

print(f"BM25: retrieved {len(bm25_docs)} docs, time {bm25_time:.4f}s")
print(f"HNSW: retrieved {len(hnsw_docs)} docs, time {hnsw_time:.4f}s")

# ----------------------------
# 4) Overlap
# ----------------------------
# BM25 doesn't provide stable IDs by default.
# We'll use (source, chunk index, or a hash) as an approximate identifier.

def doc_key(d: Document) -> str:
    md = d.metadata or {}
    src = md.get("source") or md.get("path") or md.get("file_path") or ""
    # fallback: hash content prefix
    return f"{src}::{hash(d.page_content[:2000])}"

bm25_keys = {doc_key(d) for d in bm25_docs}
hnsw_keys = {doc_key(d) for d in hnsw_docs}

inter = bm25_keys & hnsw_keys
union = bm25_keys | hnsw_keys
print(f"Overlap: {len(inter)}/{len(union)}")

# Optional: inspect results
print("\n--- BM25 top results ---")
for i, d in enumerate(bm25_docs, 1):
    text = d.page_content[:200].replace("\n", " ")
    print(f"[{i}] {text}...")

print("\n--- HNSW top results ---")
for i, d in enumerate(hnsw_docs, 1):
    dist = (d.metadata or {}).get("hnsw_distance")
    text = d.page_content[:200].replace("\n", " ")
    if dist is None:
        print(f"[{i}] {text}...")
    else:
        print(f"[{i}] dist={dist:.4f} {text}...")


BM25 time:  3.88 ms
FAISS time: 783.85 ms
HNSW time:  1070.36 ms

BM25 Results (count=2)
--------------------------------------------------------------------------------
[1] source=/Users/linghuang/Git/LLMs/LLM-RAG/09/data.txt
[10] This was the first instance of what is now a familiar experience, and so was what happened next, when I read the comments and found they were full of angry people. How could I claim that Lisp was better than other languages? Weren't they all Turing complete? People who see the responses to essays I write sometimes tell me how sorry they feel for me, but I'm not exaggerating when I reply that it has always been like this, since the very beginning. It comes with the territory. An essay must t...
--------------------------------------------------------------------------------
[2] source=/Users/linghuang/Git/LLMs/LLM-RAG/09/data.txt
I picked orange as our color partly because it's the warmest, and partly because no VC used it. In 2005 all the VCs used staid col

### 2.4 指标对比：BM25 vs FAISS vs HNSW

为了更直观地理解不同引擎在实际应用中的表现差异，我们统一在 10k 条 384 维数据上测试以下指标：
- Recall@5 (R@5)：前5条结果中是否包含正确答案的比例，衡量召回精度 。
- P50延迟 (毫秒)：50%的查询请求的响应时间低于或等于该值，代表平均查询耗时 。
- RAM占用 (MB)：索引所占内存大小，影响硬件成本和系统扩展性 。

In [30]:
import os
import time
from typing import List, Tuple, Optional

from dotenv import load_dotenv
load_dotenv()

import numpy as np
import hnswlib

from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.retrievers import BM25Retriever
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document


# ----------------------------
# Optional: stemming for BM25
# ----------------------------
try:
    import Stemmer  # PyStemmer
    _stemmer = Stemmer.Stemmer("english")
except Exception:
    _stemmer = None


# ----------------------------
# Helpers (keep your style)
# ----------------------------
def preview(doc: Document, n: int = 500) -> str:
    text = (doc.page_content or "").replace("\n", " ").strip()
    return text[:n] + ("..." if len(text) > n else "")

def print_results(title: str, docs: List[Document], n_preview: int = 500) -> None:
    print(f"\n{title} (count={len(docs)})")
    for i, d in enumerate(docs, 1):
        src = (d.metadata or {}).get("source", "")
        print("-" * 80)
        print(f"[{i}] source={src}")
        print(preview(d, n=n_preview))

def retrieve(retriever, query: str) -> List[Document]:
    # LangChain-version-safe retrieval
    try:
        return retriever.invoke(query)  # newer
    except Exception:
        return retriever.get_relevant_documents(query)  # older


# ----------------------------
# HNSW retriever (LangChain Document-compatible)
# ----------------------------
class SimpleHNSWRetriever:
    """
    HNSW retriever for LangChain Documents.
    Uses the same Embeddings object as FAISS so results are comparable.
    space='cosine' => distance = 1 - cosine_similarity (lower is better)
    """

    def __init__(
        self,
        docs: List[Document],
        embeddings: OpenAIEmbeddings,
        top_k: int = 2,
        M: int = 16,
        ef_construction: int = 100,
        ef_search: int = 50,
        embed_dim: int = 1536,
    ):
        self.docs = docs
        self.embeddings = embeddings
        self.top_k = top_k
        self.embed_dim = embed_dim

        self.id_to_doc = {i: d for i, d in enumerate(docs)}

        texts = [d.page_content for d in docs]
        vecs = embeddings.embed_documents(texts)
        vecs = np.asarray(vecs, dtype=np.float32)

        if vecs.ndim != 2 or vecs.shape[1] != embed_dim:
            raise ValueError(f"Expected embeddings shape (N, {embed_dim}), got {vecs.shape}")

        self.index = hnswlib.Index(space="cosine", dim=embed_dim)
        self.index.init_index(max_elements=len(docs), M=M, ef_construction=ef_construction)
        self.index.add_items(vecs, np.arange(len(docs)))
        self.index.set_ef(ef_search)

    def get_relevant_documents(self, query: str) -> List[Document]:
        q = np.asarray([self.embeddings.embed_query(query)], dtype=np.float32)  # (1, dim)
        ids, dists = self.index.knn_query(q, k=self.top_k)

        out: List[Document] = []
        for idx, dist in zip(ids[0].tolist(), dists[0].tolist()):
            doc = self.id_to_doc[idx]
            meta = dict(doc.metadata or {})
            meta["hnsw_distance"] = float(dist)  # 1 - cosine_sim
            out.append(Document(page_content=doc.page_content, metadata=meta))
        return out


# ----------------------------
# 1) Load
# ----------------------------
file_path = os.path.join(os.getcwd(), "data.txt")
documents = TextLoader(file_path, encoding="utf-8").load()

# ----------------------------
# 2) Split
# ----------------------------
splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=50)
docs = splitter.split_documents(documents)

# ----------------------------
# 3) BM25
# ----------------------------
# Some LC versions accept stemmer=..., some don't. We'll try and fallback.
try:
    bm25_retriever = BM25Retriever.from_documents(docs, k=2, stemmer=_stemmer)
except TypeError:
    bm25_retriever = BM25Retriever.from_documents(docs, k=2)

# ----------------------------
# 4) FAISS (your working version)
# ----------------------------
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")
vectorstore = FAISS.from_documents(docs, embeddings)
faiss_retriever = vectorstore.as_retriever(search_kwargs={"k": 2})

# ----------------------------
# 5) HNSW
# ----------------------------
hnsw_retriever = SimpleHNSWRetriever(
    docs=docs,
    embeddings=embeddings,
    top_k=2,
    embed_dim=1536,  # text-embedding-3-small dim
)

# ----------------------------
# 6) Query + timing
# ----------------------------
query = "What happened at Viaweb and Interleaf?"

t0 = time.time()
bm25_results = retrieve(bm25_retriever, query)
bm25_time = (time.time() - t0) * 1000

t0 = time.time()
faiss_results = retrieve(faiss_retriever, query)
faiss_time = (time.time() - t0) * 1000

t0 = time.time()
hnsw_results = retrieve(hnsw_retriever, query)
hnsw_time = (time.time() - t0) * 1000

print(f"\nBM25 time:  {bm25_time:.2f} ms")
print(f"FAISS time: {faiss_time:.2f} ms")
print(f"HNSW time:  {hnsw_time:.2f} ms")

print_results("BM25 Results", bm25_results)
print_results("FAISS Results", faiss_results)
print_results("HNSW Results", hnsw_results)

# ----------------------------
# 7) Scores
# ----------------------------
# FAISS distances (lower = more similar) via LC API:
faiss_with_scores: List[Tuple[Document, float]] = vectorstore.similarity_search_with_score(query, k=2)
if faiss_with_scores:
    avg_dist = sum(dist for _, dist in faiss_with_scores) / len(faiss_with_scores)
    print(f"\nFAISS average L2 distance: {avg_dist:.4f}")
    for i, (doc, dist) in enumerate(faiss_with_scores, 1):
        src = (doc.metadata or {}).get("source", "")
        print(f"  [{i}] dist={dist:.4f} source={src}")

# HNSW distance stored in metadata:
if hnsw_results:
    avg_hnsw = sum((d.metadata or {}).get("hnsw_distance", 0.0) for d in hnsw_results) / len(hnsw_results)
    print(f"\nHNSW average cosine-distance (1 - cos_sim): {avg_hnsw:.4f}")
    for i, d in enumerate(hnsw_results, 1):
        dist = (d.metadata or {}).get("hnsw_distance", None)
        text = preview(d, n=120)
        print(f"  [{i}] dist={dist:.4f} {text}")



BM25 time:  0.90 ms
FAISS time: 503.81 ms
HNSW time:  126.83 ms

BM25 Results (count=2)
--------------------------------------------------------------------------------
[1] source=/Users/linghuang/Git/LLMs/LLM-RAG/09/data.txt
[10] This was the first instance of what is now a familiar experience, and so was what happened next, when I read the comments and found they were full of angry people. How could I claim that Lisp was better than other languages? Weren't they all Turing complete? People who see the responses to essays I write sometimes tell me how sorry they feel for me, but I'm not exaggerating when I reply that it has always been like this, since the very beginning. It comes with the territory. An essay must t...
--------------------------------------------------------------------------------
[2] source=/Users/linghuang/Git/LLMs/LLM-RAG/09/data.txt
I picked orange as our color partly because it's the warmest, and partly because no VC used it. In 2005 all the VCs used staid colo

### 代码说明
1. 数据准备 ：使用Sentence-BERT生成384维嵌入向量，创建10k条测试文档和100个查询样本
2. 引擎实现 ：
   - BM25：使用LlamaIndex的BM25Retriever
   - FAISS：使用FlatL2索引（精确搜索）
   - HNSW：使用hnswlib实现近似最近邻搜索
3. 指标测量 ：
   - Recall@5 ：通过检查真实文档ID是否出现在前5条结果中计算
   - P50延迟 ：运行100次查询后取响应时间的中位数
   - RAM占用 ：使用memory_profiler测量索引初始化时的内存峰值
4. 结果可视化 ：使用matplotlib展示三种引擎在各项指标上的对比

In [31]:
import time
import random
import numpy as np
from sentence_transformers import SentenceTransformer

import faiss
import hnswlib

from langchain_core.documents import Document
from langchain_community.retrievers import BM25Retriever


# ---------------------------------
# Fast benchmark configuration
# ---------------------------------
embedding_dim = 384
num_documents = 20   # Small dataset for fast testing
num_queries = 1      # Run only one query
top_k = 5


# ---------------------------------
# 1) Lightweight embedding model
# ---------------------------------
# all-MiniLM-L6-v2 is fast to load and compute
embed_model = SentenceTransformer("all-MiniLM-L6-v2")


# ---------------------------------
# 2) Minimal test data
# ---------------------------------
texts = [f"Test doc {i}: Fast retrieval test." for i in range(num_documents)]
docs = [Document(page_content=t, metadata={"doc_id": i}) for i, t in enumerate(texts)]

# Precompute document embeddings (float32 for FAISS / HNSW)
doc_embeddings = embed_model.encode(texts).astype("float32")

# Randomly pick one document as the query
query_indices = [random.randint(0, num_documents - 1)]
queries = [texts[i] for i in query_indices]
query_embeddings = embed_model.encode(queries).astype("float32")


# ---------------------------------
# 3) Unified engine wrapper
# ---------------------------------
class FastRetrievalEngine:
    """
    Simple wrapper to standardize:
    - index building
    - query execution
    """
    def __init__(self, name, build_func, search_func):
        self.name = name
        self.build = build_func
        self.search = search_func
        self.index = None


# ---------------------------------
# BM25 (LangChain)
# ---------------------------------
def bm25_build():
    # LangChain BM25 builds directly from documents
    return BM25Retriever.from_documents(docs, k=top_k)

def bm25_search(retriever, query: str):
    start = time.time()
    try:
        out_docs = retriever.invoke(query)   # Newer LangChain API
    except Exception:
        out_docs = retriever.get_relevant_documents(query)  # Older API

    latency_ms = (time.time() - start) * 1000
    indices = [(d.metadata or {}).get("doc_id") for d in out_docs]
    return indices, latency_ms


# ---------------------------------
# FAISS (FlatL2, no training)
# ---------------------------------
def faiss_build():
    index = faiss.IndexFlatL2(embedding_dim)
    index.add(doc_embeddings)
    return index

def faiss_search(index, q_emb: np.ndarray):
    start = time.time()
    _, indices = index.search(
        q_emb.reshape(1, -1).astype("float32"),
        top_k
    )
    latency_ms = (time.time() - start) * 1000
    return indices[0].tolist(), latency_ms


# ---------------------------------
# HNSW (low-complexity configuration)
# ---------------------------------
def hnsw_build():
    index = hnswlib.Index(space="l2", dim=embedding_dim)
    index.init_index(
        max_elements=len(doc_embeddings),
        ef_construction=20,  # Lower build cost
        M=8,                 # Lower graph degree
    )
    index.add_items(doc_embeddings, np.arange(len(doc_embeddings)))
    index.set_ef(10)        # Lower search cost
    return index

def hnsw_search(index, q_emb: np.ndarray):
    start = time.time()
    indices, _ = index.knn_query(
        q_emb.reshape(1, -1).astype("float32"),
        k=top_k
    )
    latency_ms = (time.time() - start) * 1000
    return indices[0].tolist(), latency_ms


# ---------------------------------
# 4) Initialize engines
# ---------------------------------
engines = [
    FastRetrievalEngine("BM25", bm25_build, bm25_search),
    FastRetrievalEngine("FAISS", faiss_build, faiss_search),
    FastRetrievalEngine("HNSW", hnsw_build, hnsw_search),
]


# ---------------------------------
# 5) Build indexes (measure build time)
# ---------------------------------
build_times = {}
for engine in engines:
    t0 = time.time()
    engine.index = engine.build()
    build_times[engine.name] = (time.time() - t0) * 1000


# ---------------------------------
# 6) Execute query
# ---------------------------------
total_start = time.time()

query = queries[0]
q_emb = query_embeddings[0]
true_idx = query_indices[0]

results = {}
for engine in engines:
    if engine.name == "BM25":
        indices, latency = engine.search(engine.index, query)
    else:
        indices, latency = engine.search(engine.index, q_emb)

    results[engine.name] = {
        "recall": 1 if true_idx in indices else 0,
        "latency": latency,
        "build_time": build_times[engine.name],
    }

total_time = (time.time() - total_start) * 1000


# ---------------------------------
# 7) Print results table
# ---------------------------------
print("| Engine | Build (ms) | Query (ms) | Recall@5 | Total (ms) |")
print("|--------|------------|------------|----------|------------|")
for name in ["BM25", "FAISS", "HNSW"]:
    r = results[name]
    print(
        f"| {name:<6} | {r['build_time']:<10.0f} | {r['latency']:<10.0f} | "
        f"{r['recall']:<8} | {total_time:<10.0f} |"
    )


| Engine | Build (ms) | Query (ms) | Recall@5 | Total (ms) |
|--------|------------|------------|----------|------------|
| BM25   | 0          | 0          | 1        | 1          |
| FAISS  | 0          | 1          | 1        | 1          |
| HNSW   | 1          | 0          | 1        | 1          |


| 引擎       | R@5  | P50 (ms) | RAM (MB) | 备注                          |
|------------|------|----------|----------|-------------------------------|
| BM25       | 0.61 | 8        | 30       | 结果可解释，适合关键词匹配    |
| FAISS Flat | 0.89 | 45       | 150      | 暴力搜索，召回最高            |
| FAISS IVF  | 0.85 | 12       | 15       | 召回略低，但速度快            |
| HNSW       | 0.88 | 7        | 60       | ef_search 可调，性能均衡      |

# 三、一条公式做决策：R = f(数据量, 延迟, 预算)

我们可以将引擎选型简化为一个经验公式：
R = f(数据量, 延迟, 预算) 

即根据你的业务规模（数据量）、响应要求（延迟）和资源投入（预算），做出最合适的检索系统架构设计。

| 数据量范围         | 推荐方案                                                                 |
|--------------------|--------------------------------------------------------------------------|
| <10 万条           | 直接使用 BM25 + 拼写纠错即可，无需引入向量检索，节省开发成本。             |
| 10 万～100 万条     | 优先考虑 HNSW（内存充足时）或 FAISS IVF + SQ8（内存受限时）。              |
| 100 万～1000 万条   | 使用 FAISS IVF4096 + OPQ16，GPU 训练约 30 分钟，兼顾召回与效率。            |
| >1000 万条         | 建议部署 Milvus Distributed + 分层索引策略（如 IVF 做一级，HNSW 做二级）。 |

选型思路 ： 
- 小数据冷启动优先选择 BM25；
- 中等规模且强调语义匹配，HNSW 是首选；
- 超大规模、预算有限但接受一定精度损失时，FAISS IVF/PQ 更具性价比；
- 对准确性和可解释性都有要求的垂直领域，推荐使用 Hybrid 混合架构。

## 四、避坑指南

以下是我们在多个项目中总结出的“血泪教训”，助你在落地过程中少走弯路。

- 维度不是越高越好 ：MiniLM 384 维足以应对大多数任务，盲目追求 1024 维只会浪费内存。
- 建库前务必 shuffle 数据 ：顺序写入会导致 IVF 聚类倾斜，查询时退化为线性扫描。
- 监控索引健康状态 ：Milvus 提供 Prometheus 指标 cache_hit_ratio，低于 80% 应立即扩容。
- 长文本切片要谨慎 ：一刀切 512 字可能截断关键信息；推荐滑动窗口（256/128）。
- 冷启动没数据怎么办 ：使用 HyDE 生成伪文档（第13讲）或先用 BM25 托底。
- 合规红线不能碰 ：金融、医疗等敏感领域禁止明文出境，优先选用私有化部署。

## 五、小结
综上所述，在 RAG 初期选型中：
- BM25 虽然不擅长捕捉语义相似性，但在关键词主导、冷启动或需解释性的场景中依然不可替代 
- FAISS IVF 在大规模部署中更具成本优势 ，适合 GPU 预算有限的项目；
- HNSW 是兼顾高召回与低延迟的理想选择 ，特别适用于需要实时语义检索的场景；
各有优点，下一讲我们来聊聊如何结合他们的优点实现混合检索优化。